In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125372")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FHR558FZC to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-125372
Azure region: southcentralus
Subscription id: 311ae77a-da21-4c71-9058-d2f6f0fcb3f3
Resource group: aml-quickstarts-125372


In [2]:
import os
#files and directories
cc=os.getcwd
os.listdir(os.curdir)  

['.ipynb_aml_checkpoints', 'outputs', 'train.py', 'udacity-project.ipynb']

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_cluster_name = "ompute-target"

# Verify that cluster does not exist already
try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [10]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling({"--C": uniform(0.05, 2), "--max_iter": choice(range(10,200,20)) })

# Specify a Policy
### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, slack_amount=None, delay_evaluation=0)

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
est = SKLearn(source_directory =".", compute_target=compute_cluster, entry_script="train.py")


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling = ps, 
    primary_metric_name ='Accuracy', 
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs = 12, 
    max_concurrent_runs=4, 
    policy=policy, 
    estimator=est
)

In [11]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [13]:
best_run=hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_files = best_run.get_file_names()

print('Bets Run ID', best_run.id)
print('\n Accuracy', best_run_metrics['Accuracy'])

Bets Run ID HD_c8899b31-ba4a-4e13-bb12-f86d594c4fae_0

 Accuracy 0.9072837632776934


In [14]:
import joblib
# Get your best run and save the model from that run.

best_model = best_run.register_model(model_name='model_log_hd', model_path='outputs/model_hd.joblib')

In [15]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path_to_csv = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

ds = TabularDatasetFactory.from_delimited_files(path=path_to_csv)

In [16]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train['y']=y_train
x_test['y']=y_test


x_train.head()


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
26465,52,1,0,1,0,8,3,13,14,999,...,0,0,0,0,1,0,0,0,0,0
1387,41,1,0,1,0,6,4,84,3,999,...,1,1,0,0,0,0,0,0,0,0
22628,27,0,0,0,0,7,1,331,2,999,...,0,0,0,0,0,0,1,0,0,0
8336,58,1,0,1,0,6,2,137,3,999,...,1,1,0,0,0,0,0,0,0,0
19898,36,0,0,1,0,5,1,143,4,999,...,1,1,0,0,0,0,0,0,0,0


In [18]:
columns=x_train.columns
x_train.to_csv(path_or_buf='trainset.csv', columns=columns, header=True, index=False) 
x_test.to_csv(path_or_buf='testset.csv', columns=columns, header=True, index=False) 

In [19]:
from azureml.data import TabularDataset 
datastore=ws.get_default_datastore() 
datastore.upload_files(['trainset.csv'])

Uploading an estimated of 1 files
Uploading trainset.csv
Uploaded trainset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [20]:
train=TabularDatasetFactory.from_delimited_files([(datastore, 'trainset.csv')])

datastore.upload_files(['testset.csv'])

Uploading an estimated of 1 files
Uploading testset.csv
Uploaded testset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [21]:
test=TabularDatasetFactory.from_delimited_files([(datastore, 'testset.csv')])

In [22]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
task="classification" 
automl_config = AutoMLConfig( 
    experiment_timeout_minutes=30, 
    task=task, 
    primary_metric='accuracy', 
    training_data=train, 
    validation_data = test, 
    label_column_name='y', 
    compute_target=compute_cluster 
)

In [23]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)
RunDetails(automl_run).show()

Running on remote.
Running on remote compute: ompute-target
Parent Run ID: AutoML_3e13f073-0625-42e3-8aa8-ee14e805b2c0

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of th

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [24]:
# Retrieve and save your best automl model.
automl_run, fitted_automl_model = automl_run.get_output() 
print(fitted_automl_model) #To print
import joblib  
joblib.dump(fitted_automl_model, "fitted_automl_model.joblib")

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(allow_chargram=None, enable_dnn=None,
                                 enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('MaxAbsScale...
                                   colsample_bylevel=1, colsample_bynode=1,
                                   colsample_bytree=1, gamma=0,
                                   learning_rate=0.1, max_delta_step=0,
                                   max_depth=3, min_child_weight=1, missing=nan,
                                   n_estimators=100, n_jobs=1, nt

['fitted_automl_model.joblib']

In [25]:
print('Bets Run ID', automl_run.id)

Bets Run ID AutoML_3e13f073-0625-42e3-8aa8-ee14e805b2c0_1
